# MLE and errors

In [4]:
import numpy as np
from scipy import integrate, stats, interpolate
from toolbox import vis
from toolbox import as_toolbox as ast
import urllib
import matplotlib.pyplot as plt
import importlib as il
from iminuit import Minuit
import toolbox
from scipy import optimize
from toolbox.ExternalFunctions import UnbinnedLH

In [7]:
il.reload(toolbox)

<module 'toolbox' (namespace)>

# Ex. 1 

# Ex. 2 